In [ ]:
import dagshub
import mlflow
dagshub.init(repo_owner='AMR-ITH', repo_name='yt-comment-analyzer', mlflow=True)

⠼ Waiting for authorization

Accessing as AMR-ITH

Initialized MLflow to track repo "AMR-ITH/yt-comment-analyzer"

Repository AMR-ITH/yt-comment-analyzer initialized!

In [ ]:
# Set or create an experiment
mlflow.set_experiment("ML Algos with HP Tuning")

<Experiment: artifact_location='mlflow-artifacts:/6372c8dc1c894e3290997ce5e2b55158', creation_time=1751625225992, experiment_id='4', last_update_time=1751625225992, lifecycle_stage='active', name='ML Algos with HP Tuning', tags={}>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import mlflow
import mlflow.sklearn
import optuna
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
df = pd.read_csv('/content/reddit_preprocessing.csv')
df.head()

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1


In [ ]:
df = pd.read_csv('/content/reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [ ]:
# Step 1: Clean data
df = df.dropna(subset=['category'])
y = df['category']
X_raw = df['clean_comment']

# Step 2: Train-test split BEFORE vectorization or resampling
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X_raw, y, test_size=0.2, random_state=42, stratify=y)

# Step 3: Vectorization only on training data
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=3000)
X_train_vect = vectorizer.fit_transform(X_train_raw)
X_test_vect = vectorizer.transform(X_test_raw)  # Note: transform only!

# Step 4: Undersampling (only on training data)
rus = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train_vect, y_train)

# # Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_train_resampled, y_train_resampled, test_size=0.2, random_state=42, stratify=y_train_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_RandomUnderSampler_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)



# Step 6: Optuna objective function for Logistic Regression
def objective_logreg(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])

    # LogisticRegression model setup with balanced class weight
    model = LogisticRegression(C=C, penalty=penalty, solver='liblinear', random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for Logistic Regression, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_logreg, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = LogisticRegression(C=best_params['C'], penalty=best_params['penalty'], solver='liblinear', random_state=42)

    # Log the best model with MLflow, passing the algo_name as "LogisticRegression"
    log_mlflow("LogisticRegression", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for Logistic Regression
run_optuna_experiment()


[I 2025-07-11 12:43:10,303] A new study created in memory with name: no-name-5985c4fd-67cc-49a0-9a46-bb52ea2e02ec
[I 2025-07-11 12:43:10,453] Trial 0 finished with value: 0.8181358929022481 and parameters: {'C': 2.975773259308346, 'penalty': 'l2'}. Best is trial 0 with value: 0.8181358929022481.
[I 2025-07-11 12:43:10,485] Trial 1 finished with value: 0.48901237686284416 and parameters: {'C': 0.026086223346419794, 'penalty': 'l1'}. Best is trial 0 with value: 0.8181358929022481.
[I 2025-07-11 12:43:10,551] Trial 2 finished with value: 0.6855266481434705 and parameters: {'C': 0.03176105735783011, 'penalty': 'l2'}. Best is trial 0 with value: 0.8181358929022481.
[I 2025-07-11 12:43:10,610] Trial 3 finished with value: 0.7552412225309422 and parameters: {'C': 0.2296330899283873, 'penalty': 'l1'}. Best is trial 0 with value: 0.8181358929022481.
[I 2025-07-11 12:43:10,631] Trial 4 finished with value: 0.33316494064157615 and parameters: {'C': 0.0006020930239735948, 'penalty': 'l1'}. Best is

🏃 View run LogisticRegression_RandomUnderSampler_TFIDF_Trigrams at: https://dagshub.com/AMR-ITH/yt-comment-analyzer.mlflow/#/experiments/4/runs/12bef1504f9a454e97041b59ec68d7d7
🧪 View experiment at: https://dagshub.com/AMR-ITH/yt-comment-analyzer.mlflow/#/experiments/4
